In [7]:
import nltk
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt
import re
import seaborn as sns
from IPython.display import display
pd.options.mode.chained_assignment = None
import matplotlib
matplotlib.style.use('ggplot')


#### Enter handler you want to read

In [50]:
handler = input().strip()
tweets = pd.read_csv('%s_tweets.csv' % handler, encoding = "utf-8")
tweets.head()

i_ajinkya


,id,created_at,text
0,813256267737153538,2016-12-26 05:32:58,"b'@idea_cares connection is suspended, not clo..."
1,812646930430001152,2016-12-24 13:11:41,b'@DoT_India @manojsinhabjp @ideacellular Broa...
2,812646079053189121,2016-12-24 13:08:18,b'@DoT_India user id 012373020264.'
3,811978714934349826,2016-12-22 16:56:26,b'@idea_cares Done.'
4,811976877929861120,2016-12-22 16:49:08,b'@DoT_India @manojsinhabjp @ideacellular #DOT...


In [51]:
tweets['text'] = tweets['text'].apply(lambda x: str(x))
tweets.head()

,id,created_at,text
0,813256267737153538,2016-12-26 05:32:58,"b'@idea_cares connection is suspended, not clo..."
1,812646930430001152,2016-12-24 13:11:41,b'@DoT_India @manojsinhabjp @ideacellular Broa...
2,812646079053189121,2016-12-24 13:08:18,b'@DoT_India user id 012373020264.'
3,811978714934349826,2016-12-22 16:56:26,b'@idea_cares Done.'
4,811976877929861120,2016-12-22 16:49:08,b'@DoT_India @manojsinhabjp @ideacellular #DOT...


In [15]:
import re
#Preprocessing del RT @blablabla:
tweets['text_new'] = ''
tweets['tweetos'] = '' 

#add tweetos first part
for i in range(len(tweets['text'])):
    try:
        tweets['tweetos'][i] = tweets['text'].str.split(':')[i][0]
    except AttributeError:    
        tweets['tweetos'][i] = 'other'

#Preprocessing tweetos. select tweetos contains 'RT @'
for i in range(len(tweets['text'])):
    if tweets['tweetos'].str.contains('RT @')[i]  == False:
        tweets['tweetos'][i] = 'other'

#'text_new' is the feature 'text' without the tweetos    
for i in range(len(tweets['text'])):
    m = re.search('(?<=:)(.*)', tweets['text'][i])
    if tweets['text'].str.contains('RT @')[i]  == True:
        try:
            tweets['text_new'][i]=m.group(0)
        except AttributeError:
            tweets['text_new'][i]=tweets['text'][i] 
    else:       
        tweets['text_new'][i] =  tweets['text'][i] 

In [18]:
tweets['tweetos'].head()

0    b'RT @analyticbridge
1    b'RT @analyticbridge
2        b'RT @KirkDBorne
3         b'RT @AndrewYNg
4         b'RT @pydatasci
Name: tweetos, dtype: object

In [17]:
tweets['text_new'].head()

0     When Does Deep Learning Work Better Than SVMs...
1     49 Machine Learning Resources and Related Art...
2     BIG CheatSheet on #MachineLearning #Algorithm...
3     Interested in Deep Reinforcement Learning? Gr...
4     The Python Data Science Handbook is now avail...
Name: text_new, dtype: object